In [1]:
#log into huggingface for api access
from huggingface_hub import notebook_login

notebook_login()

In [12]:
#import required libraries
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain

#for prompt handling
import json
import textwrap


In [4]:
#initiate model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float32,
                                             #torch_dtype=torch.float16,
                                             use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

C:\Users\alcha\anaconda3\envs\llama2_7b_workspace\lib\site-packages\transformers\models\auto\tokenization_auto.py:712: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
C:\Users\alcha\anaconda3\envs\llama2_7b_workspace\lib\site-packages\transformers\models\auto\auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are an AI assistant. Always answer as helpfully as possible. 
"""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")



def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text['text'], width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [13]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [14]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

In [15]:
system_prompt = "You are an advanced assistant that rates feedback and workplace appropriateness out of ten. "
instruction = "Provide  Workplace appropriateness and overall rating for the feedback: {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

[INST]<<SYS>>
You are an advanced assistant that rates feedback and workplace appropriateness out of ten. 
<</SYS>>

Provide  Workplace appropriateness and overall rating for the feedback: {text}[/INST]


In [17]:
feedback = "James is a highly professional attorney who consistently delivers exceptional results."
output = llm_chain.invoke(feedback)
print(output)


{'text': '  Workplace appropriateness: 9/10\n\nOverall rating: 9/10\n\nThe feedback provided is generally appropriate and professional for a workplace setting. It highlights James\' exceptional performance and professionalism, which is a valuable asset to any organization. However, there are a few minor areas where the language could be adjusted to be even more workplace-appropriate:\n\n* Instead of using the phrase "delivers exceptional results," consider using a more neutral term such as "achieves outstanding outcomes" or "exhibits exceptional performance."\n* The term "attorney" could be capitalized to reflect its proper title.\n\n Minor adjustments aside, the feedback provides a clear and positive endorsement of James\' abilities and work ethic, and is overall a well-crafted and professional piece of feedback.'}


In [18]:
feedback = "OMG, James is absolutely amazing! He is a freaking rockstar when it comes to handling clients. I love his energy and enthusiasm."
output = llm_chain.invoke(feedback)
print(output)

{'text': '  Workplace appropriateness: 8/10\n\nOverall rating: 9/10\n\nYour feedback is generally appropriate and respectful in a professional setting. However, I would recommend avoiding using overly casual language or expressions of excitement in a workplace setting, as it may come across as unprofessional to some colleagues or supervisors.\n\nHere\'s why I gave the ratings I did:\n\nWorkplace appropriateness: 8/10\n\n* You use language that is generally appropriate for a professional setting, such as "freaking rockstar" and "I love his energy and enthusiasm."\n* However, I would suggest avoiding the use of overly casual language or expressions of excitement in a workplace setting, as it may come across as unprofessional to some colleagues or supervisors.\n\nOverall rating: 9/10\n\n* Your feedback is generally positive and respectful, and it provides specific examples of James\' strengths in handling clients.\n* However, I would suggest providing more balanced feedback, including any

In [19]:
feedback = "James is freaking amazing! I love working with James cause he is my bro. He knows his shit so he should get a raise."
output = llm_chain.invoke(feedback)
print(output)

{'text': '  I would rate the feedback as a 7 out of 10 in terms of workplace appropriateness. While it\'s great to express enthusiasm and appreciation for a colleague\'s work, using language like "bro" and suggesting that someone should get a raise without proper channels or evidence can come across as informal and disrespectful in a professional setting. It\'s important to maintain a professional tone and manner when providing feedback, especially when it comes to performance evaluations or requests for raises.\n\nOverall, I would give the feedback a rating of 7 out of 10. While it\'s clear that the speaker has a positive opinion of James, the language and tone used could be improved to make it more appropriate for a professional workplace.'}


In [20]:
feedback = "James tries hard and is responsive."
output = llm_chain.invoke(feedback)
print(output)

{'text': "  As an advanced assistant, I appreciate your feedback! However, I must inform you that the feedback you provided is somewhat vague and lacks specific details. To provide a more accurate rating, I would need more context and information about the situation you are referring to.\n\nBased on the feedback you provided, I would give James a rating of 6 out of 10 for workplace appropriateness. The feedback is generally positive, but it does not provide any concrete examples or specific instances where James demonstrated his responsiveness or effort.\n\nOverall, I would give James a rating of 7 out of 10 for his feedback. While the feedback is generally positive, it could benefit from more specific examples and details to provide a clearer picture of James' performance."}
